# CatBoost

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_squared_error
from catboost import CatBoostClassifier, CatBoostRegressor, Pool



In [4]:
#load

csv_path = "./data/train.csv"
df = pd.read_csv(csv_path)
df.head

<bound method NDFrame.head of         time         A         B         C         D         E         F  \
0          0  0.207366 -0.159951 -0.634176 -0.580962 -0.266505  0.060173   
1          1  0.188828 -0.265508  0.042143 -0.550442 -0.132319 -0.185219   
2          2 -0.144261 -0.577142 -0.214634 -0.747391 -0.184255 -0.464831   
3          3  0.208982 -0.310449  0.513708 -0.562868  0.742308 -0.305487   
4          4  0.093320 -0.358156  0.173188 -0.687296 -0.161461 -0.116062   
...      ...       ...       ...       ...       ...       ...       ...   
79995  80000 -0.339802 -0.372094 -0.120952 -0.363438  0.266299 -0.222866   
79996  80001 -0.421921 -0.222554 -0.689585 -0.512266 -0.206562 -0.494450   
79997  80002 -0.467038 -0.282018  0.010707 -0.589594  0.067635 -0.410820   
79998  80003 -0.369710 -0.322384 -0.540326 -0.563206 -0.568736 -0.550024   
79999  80004 -0.304899 -0.047375 -0.932102 -0.229424 -0.474211 -0.336000   

              G         H         I         J         K  

In [ ]:
y_col = "y1"
X = df.drop(columns=[y_col])
y = df[y_col]